In [14]:
import re
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from konlpy.tag import Okt, Kkma, Komoran

In [2]:
data = pd.read_csv('./data/naver_movie_dataset.csv') # 데이터 불러오기

In [3]:
data.head(3) # 데이터 확인

,Unnamed: 0,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,0,100% 울프: 푸들이 될 순 없어,애니메이션,2020,12.24,9.50,16,유서 깊은 늑대인간 가문의 후계자 프레디는 자랑스러운 아버지의 뒤를 이어 위기에 빠...,NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20201208_299/1...
1,1,108명의 여자와 섹스하는 방법,코미디|드라마,2020,5.21,1.00,1,유명 시나리오 작가인 '카이바 고로'는 SNS 통해 아내 '아야코'의 외도 사실을 ...,마츠오 스즈키|나카야마 미호,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200512_12/15...
2,2,16세의 사운드트랙,코미디|멜로/로맨스,2020,7.02,7.75,4,“축하는 개뿔! 이제 나만 남았잖아 쪽팔리게!!!” - 유일한 키스 無경험자 ‘메이...,스칼렛 마샬|제임스 콜로웨이,자멜 하디쿠라|잭볼|숀 미칼레프,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200619_172/1...


In [4]:
data.info() # 데이터 확인 줄거리에서 null값이 있는 것은 4개 뿐이다. 배우 정보를 재외하면 널값이 없다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15028 entries, 0 to 15027
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  15028 non-null  int64  
 1   title       15028 non-null  object 
 2   genre       15028 non-null  object 
 3   year        15028 non-null  int64  
 4   date        15028 non-null  float64
 5   rating      15028 non-null  float64
 6   vote_count  15028 non-null  int64  
 7   plot        15024 non-null  object 
 8   main_act    14502 non-null  object 
 9   supp_act    11973 non-null  object 
 10  page_url    15028 non-null  object 
 11  img_url     15028 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.4+ MB


In [6]:
data.drop(data[data['plot'].isnull()].index, inplace=True) #줄거리 정보 없는거 삭제

In [10]:
def preprocessing(plot, okt, plot_stopwords = True):

    plot_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", plot)
    word_plot = okt.morphs(plot_text, stem=True)
    stop_words = set([])
    
    if plot_stopwords:
        word_plot = [token for token in word_plot if not token in stop_words]
    return word_plot

In [15]:
okt = Okt()
clean_plot = []

for plot in data['plot']:
    if type(plot) == str:
        clean_plot.append(preprocessing(plot, okt, plot_stopwords = False))
    else:
        clean_plot.append([])

In [16]:
clean_plot

[['유서',
  '깊다',
  '늑대인간',
  '가문',
  '의',
  '후계',
  '자',
  '프레디',
  '는',
  '자랑스럽다',
  '아버지',
  '의',
  '뒤',
  '를',
  '이다',
  '위기',
  '에',
  '빠지다',
  '인간',
  '들',
  '을',
  '구',
  '하다',
  '임무',
  '를',
  '수행',
  '하다',
  '날',
  '만을',
  '손꼽다',
  '기다리다',
  '있다',
  '드디어',
  '꿈',
  '에',
  '그리다',
  '변신',
  '의',
  '순간',
  '늑대',
  '가',
  '아니다',
  '귀엽다',
  '푸다',
  '들이다',
  '되어다',
  '버리다',
  '프레디',
  '프레디',
  '는',
  '자신',
  '이',
  '순도',
  '늑대인간',
  '이라는',
  '것',
  '을',
  '증명',
  '하다',
  '위해',
  '가문',
  '의',
  '비밀',
  '이',
  '담기다',
  '있다',
  '보물',
  '문스톤',
  '을',
  '찾다',
  '나서다'],
 ['유명',
  '시나리오',
  '작가',
  '인',
  '카이바',
  '고로',
  '는',
  '통해',
  '아내',
  '아야',
  '코',
  '의',
  '외도',
  '사실',
  '을',
  '알',
  '게',
  '되다',
  '여전하다',
  '아내',
  '를',
  '사랑',
  '하',
  '지만',
  '가만히',
  '있다',
  '솟구치다',
  '화',
  '를',
  '차다',
  '수',
  '없다',
  '카이바',
  '고로',
  '기상',
  '천',
  '외한',
  '복수',
  '방법',
  '을',
  '고민',
  '하다',
  '그',
  '는',
  '아내',
  '의',
  '게시',
  '물',
  '에',
  '좋다',
  '요가',
  '개',
  '눌려',
  '있다',
  '것',


In [17]:
preprocess_data = pd.DataFrame({'plot' : clean_plot})
print(preprocess_data.shape)

(15024, 1)


In [18]:
preprocess_data.head()

,plot
0,"[유서, 깊다, 늑대인간, 가문, 의, 후계, 자, 프레디, 는, 자랑스럽다, 아버..."
1,"[유명, 시나리오, 작가, 인, 카이바, 고로, 는, 통해, 아내, 아야, 코, 의..."
2,"[축하, 는, 개뿔, 이제, 나, 만, 남다, 쪽팔리다, 유일하다, 키스, 경험, ..."
3,"[제, 차, 세계대전, 이, 한창, 인, 년, 독일군, 에, 의하다, 모든, 통신망..."
4,"[인류, 역사상, 최악, 의, 전쟁, 이라, 일, 컬, 어, 지는, 제, 차, 세계..."


In [20]:
sentences = []
for context in preprocess_data["plot"] :
    sentence = ' '.join(context)
    sentences.append([sentence])

preprocess_data_df = pd.DataFrame(sentences, columns=['plot'])
preprocess_data_df.tail()

,plot
15019,스페인 의 세빌랴 에 살다 투우사 지망 생 후안 가야르 도 크리스토퍼 라이 들 불다...
15020,세 의 유복하다 실업가 죠지 파파 조글루 는 성적 호기심 이 퍽 강하다 남자 다 어...
15021,필립 은 비록 바람둥이 지만 아내 메이 만은 끔찍하다 사랑 하나 하지만 메이 는 사...
15022,금광 을 찾다 알래스카 에 온 찰리 는 살인 범 블랙 의 오두막 에서 금광 을 찾다...
15023,암흑가 갱단 홍흥 사의 대부 이 곤 이 살해 되다 장례식 을 위해 미국 에서 온 그...


In [21]:
preprocess_data_df.to_csv('./data/preprocess_plot_Okt.csv')